<a href="https://colab.research.google.com/github/wellBorges/PUC_PLN/blob/main/u4_01_hands_on_1_similaridade_de_textos_pratica_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Similaridade de Textos
A similaridade de textos indica o quão "próximos" dois textos estão em significado ou na forma da escrita.


In [1]:
# instalação dos pacotes necessários
!pip install nltk==3.8.1
!pip install unidecode==1.3.8
!pip install scikit-learn==1.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.1 MB/s eta 0:00:00


In [2]:
# Importações da biblioteca padrão
import datetime
import math
import re
import string
import sys
import warnings

# Importações de bibliotecas de terceiros
import nltk

from nltk.corpus import stopwords

import sklearn
from sklearn.feature_extraction.text import CountVectorizer

from unidecode import unidecode

# Downloads do NLTK
nltk.download('punkt')
nltk.download("stopwords")

# Configurações e comandos específicos (por exemplo, desativar avisos)
warnings.filterwarnings('ignore')

print("Pacotes importados com sucesso; notebook pronto para uso!")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Pacotes importados com sucesso; notebook pronto para uso!


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# Declara define funções utilitárias utilizadas no notebook.
def formata_msg(nivel, msg):
    """
    Formata uma mensagem de log incluindo o nível de severidade, timestamp
    e a mensagem.

    Parâmetros:
    - nivel (str): Nível de severidade da mensagem (ex: 'INFO', 'ERROR', 'WARNING').
    - msg (str): A mensagem de log propriamente dita.

    Retorna:
    - str: A mensagem de log formatada.
    """
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    return f"[{nivel}] {timestamp} - {msg}"

def preprocessa_texto(texto):
    """
    Preprocessa o texto fornecido realizando várias etapas de limpeza.

    Etapas:
    1. Tokeniza o texto.
    2. Converte os tokens para minúsculos.
    3. Remove stopwords em português.
    4. Remove números dos tokens.
    5. Exclui tokens que são pontuações.
    6. Remove acentuações dos tokens.

    Parâmetros:
    texto (str): O texto a ser preprocessado.

    Retorna:
    list: Lista de tokens preprocessados.
    """

    # Tokeniza o texto usando um padrão para capturar palavras e pontuações.
    padrao = r"\w+(?:'\w+)?|[^\w\s]"
    tokens_preprocessados = re.findall(padrao, texto)

    # Converte os tokens para minúsculos para padronizar a capitalização.
    tokens_preprocessados = [token.lower() for token in tokens_preprocessados]

    # Remove stopwords para reduzir o conjunto de tokens a palavras significativas.
    portugues_stops = stopwords.words('portuguese')
    tokens_preprocessados = [token for token in tokens_preprocessados if token not in portugues_stops]

    # Remove números, pois geralmente não contribuem para o significado do texto.
    tokens_preprocessados = [re.sub(r'\d+', '', token) for token in tokens_preprocessados if re.sub(r'\d+', '', token)]

    # Exclui tokens que são pontuações, pois raramente são úteis para análise de texto.
    tokens_preprocessados = [token for token in tokens_preprocessados if token not in string.punctuation]

    # Remove acentuações para padronizar os tokens.
    tokens_preprocessados = [unidecode(token) for token in tokens_preprocessados]

    return ' '.join(tokens_preprocessados)

print(formata_msg("INFO", "Funções utilitárias prontas para utilização."))
print(formata_msg("INFO", f"Versão do Python: {sys.version} "))

[INFO] 2024-06-19 16:24:32 - Funções utilitárias prontas para utilização.
[INFO] 2024-06-19 16:24:32 - Versão do Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0] 


In [4]:
texto_1 = preprocessa_texto("O gato comeu o rato")
texto_2 = preprocessa_texto("O rato comeu a comida do gato")
print(formata_msg("INFO",f"\n{texto_1}\n{texto_2}"))

[INFO] 2024-06-19 16:24:45 - 
gato comeu rato
rato comeu comida gato


# Similaridade de Jaccard

In [5]:
def similaridade_jaccard(a, b):
  """Calcula a similaridade de Jaccard entre duas listas.

  A similaridade de Jaccard é uma medida usada para comparar a semelhança e
  diversidade de conjuntos de amostras. O coeficiente de similaridade de
  Jaccard é definido como o tamanho da interseção dividido pelo tamanho da
  união das amostras.

  Args:
    a: Uma lista de elementos.
    b: Outra lista de elementos.

  Returns:
    Um float representando a similaridade de Jaccard entre as duas listas, que
    é o tamanho da interseção dos conjuntos dividido pelo tamanho da união dos
    conjuntos.

  Raises:
    ZeroDivisionError: Se a união das listas resultar em um conjunto vazio, o
    que levaria a uma divisão por zero.
  """
  interseccao = len(set.intersection(*[set(a), set(b)]))
  uniao = len(set.union(*[set(a), set(b)]))
  return interseccao / uniao

In [6]:
corpus = [texto_1, texto_2]
tokens = [texto.split(" ") for texto in corpus]
print(formata_msg("INFO",f"\n{tokens}"))

[INFO] 2024-06-19 16:25:04 - 
[['gato', 'comeu', 'rato'], ['rato', 'comeu', 'comida', 'gato']]


In [7]:
similaridade = similaridade_jaccard(tokens[0], tokens[1])
print(formata_msg("INFO",f"\nSimilaridade de Jaccard entre os textos: {similaridade}"))

[INFO] 2024-06-19 16:25:39 - 
Similaridade de Jaccard entre os textos: 0.75


# Similaridade Euclidiana

In [8]:
def similaridade_euclidiana(a, b):
  """Calcula a similaridade euclidiana entre dois vetores.

  A similaridade euclidiana é determinada pela transformação da distância
  euclidiana entre dois pontos (vetores) em uma medida de similaridade.
  Esta transformação é realizada através da função exponencial, que mapeia a
  distância para um valor entre 0 e 1, onde valores próximos a 1 indicam alta
  similaridade e valores próximos a 0, baixa similaridade.

  Args:
    a: Uma lista ou vetor de valores numéricos.
    b: Outra lista ou vetor de valores numéricos, com a mesma dimensão de `a`.

  Returns:
    Um float representando a similaridade euclidiana entre os vetores `a` e `b`.

  Raises:
    ValueError: Se os vetores `a` e `b` têm dimensões diferentes.
  """
  if len(a) != len(b):
    raise ValueError("Os vetores 'a' e 'b' devem ter a mesma dimensão.")

  distancia = math.sqrt(sum(math.pow(x - y, 2) for x, y in zip(a, b)))
  return math.exp(-distancia)

In [9]:
vetorizador = CountVectorizer()
frequencias = vetorizador.fit_transform(corpus)
print(formata_msg("INFO",f"Tokens:\n{vetorizador.get_feature_names_out()}\n"))
print(formata_msg("INFO",f"Frequências:\n{frequencias.toarray()}"))

[INFO] 2024-06-19 16:26:02 - Tokens:
['comeu' 'comida' 'gato' 'rato']

[INFO] 2024-06-19 16:26:02 - Frequências:
[[1 0 1 1]
 [1 1 1 1]]


In [10]:
similaridade = similaridade_euclidiana(frequencias.toarray()[0], frequencias.toarray()[1])
print(formata_msg("INFO",f"\nSimilaridade Euclidiana entre os textos: {similaridade}"))

[INFO] 2024-06-19 16:26:40 - 
Similaridade Euclidiana entre os textos: 0.36787944117144233


# Similaridade de Cosseno


In [11]:
def norma_vetor(x):
  """Calcula a norma (ou magnitude) de um vetor.

  A norma é calculada como a raiz quadrada da soma dos quadrados de cada
  elemento do vetor. Esta função é comumente usada em operações de álgebra
  linear e análise de vetores.

  Args:
    x: Uma lista de valores numéricos representando um vetor.

  Returns:
    Um float representando a norma do vetor, arredondado para 3 casas decimais.
  """
  return round(math.sqrt(sum(a * a for a in x)), 3)

def similaridade_cosseno(a, b):
  """Calcula a similaridade de cosseno entre dois vetores.

  A similaridade de cosseno é uma medida que calcula o cosseno do ângulo entre
  dois vetores no espaço multidimensional, sendo usada frequentemente para
  medir a semelhança entre dois vetores. O resultado varia de -1 a 1, onde 1
  indica vetores idênticos, 0 indica ortogonalidade, e -1 indica vetores
  diametralmente opostos.

  Args:
    a: Uma lista de valores numéricos representando o primeiro vetor.
    b: Uma lista de valores numéricos representando o segundo vetor.

  Returns:
    Um float representando a similaridade de cosseno entre os vetores `a` e `b`.

  Raises:
    ValueError: Se os vetores `a` e `b` têm dimensões diferentes.
  """
  if len(a) != len(b):
    raise ValueError("Os vetores 'a' e 'b' devem ter a mesma dimensão.")

  numerador = sum(x * y for x, y in zip(a, b))
  denominador = norma_vetor(a) * norma_vetor(b)

  return numerador / float(denominador)

In [12]:
similaridade = similaridade_cosseno(frequencias.toarray()[0], frequencias.toarray()[1])
print(formata_msg("INFO",f"\nSimilaridade de Cosseno: {similaridade}"))

[INFO] 2024-06-19 16:27:21 - 
Similaridade de Cosseno: 0.8660508083140878
